In [1]:
import pyspark
sc = pyspark.SparkContext('local[*]')

In [2]:
sqlContext = pyspark.SQLContext(sc)

In [3]:
!pip install boto

    100% |████████████████████████████████| 1.4MB 880kB/s 
You are using pip version 8.1.0, however version 8.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
! aws s3 cp "s3://tweets/" ./febrero --recursive --exclude="*" --include="2016-02-*"

download: s3://opi-hdfs/raw/tweets/2016-02-01/tweets.1454284802816.tmp to febrero/2016-02-01/tweets.1454284802816.tmp
download: s3://opi-hdfs/raw/tweets/2016-02-01/tweets.1454344813269 to febrero/2016-02-01/tweets.1454344813269
download: s3://opi-hdfs/raw/tweets/2016-02-02/tweets.1454371202225.tmp to febrero/2016-02-02/tweets.1454371202225.tmp
download: s3://opi-hdfs/raw/tweets/2016-02-02/tweets.1454431212721 to febrero/2016-02-02/tweets.1454431212721
download: s3://opi-hdfs/raw/tweets/2016-02-03/tweets.1454457602374.tmp to febrero/2016-02-03/tweets.1454457602374.tmp
download: s3://opi-hdfs/raw/tweets/2016-02-03/tweets.1454517613003 to febrero/2016-02-03/tweets.1454517613003
download: s3://opi-hdfs/raw/tweets/2016-02-04/tweets.1454544001995.tmp to febrero/2016-02-04/tweets.1454544001995.tmp
download: s3://opi-hdfs/raw/tweets/2016-02-04/tweets.1454604012584 to febrero/2016-02-04/tweets.1454604012584
download: s3://opi-hdfs/raw/tweets/2016-02-05/tweets.1454630402489.tmp to febrero/2016-0

In [6]:
log_raw = sc.textFile("./marzo/2016-03-01/*")

In [35]:
log_raw_hive = sqlContext.read.json("./marzo/2016-03-01/*")

In [37]:
log_raw_hive.count()

133249

In [39]:
log_raw_hive.printSchema()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- id_str: string (nullable = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true

In [40]:
log_raw_hive.registerTempTable('txs')
#sqlContext.sql("select * from ")

In [52]:
sqlContext.sql("describe txs").show()

+--------------------+--------------------+-------+
|            col_name|           data_type|comment|
+--------------------+--------------------+-------+
|        contributors|              string|       |
|         coordinates|struct<coordinate...|       |
|          created_at|              string|       |
|            entities|struct<hashtags:a...|       |
|   extended_entities|struct<media:arra...|       |
|      favorite_count|              bigint|       |
|           favorited|             boolean|       |
|        filter_level|              string|       |
|                 geo|struct<coordinate...|       |
|                  id|              bigint|       |
|              id_str|              string|       |
|in_reply_to_scree...|              string|       |
|in_reply_to_statu...|              bigint|       |
|in_reply_to_statu...|              string|       |
| in_reply_to_user_id|              bigint|       |
|in_reply_to_user_...|              string|       |
|     is_quo

In [58]:
sqlContext.sql("select id, geo.coordinates from txs where geo is not null limit 1").show()

+------------------+--------------------+
|                id|         coordinates|
+------------------+--------------------+
|704455835645153280|[25.4336309, -100...|
+------------------+--------------------+



In [59]:
sqlContext.sql("select count(*) from txs where geo is not null").show()

+-----+
|  _c0|
+-----+
|22509|
+-----+



In [62]:
sqlContext.sql("select count(*) from txs").show()

+------+
|   _c0|
+------+
|133249|
+------+



In [63]:
sqlContext.sql("select count(*) from txs where coordinates is not null").show()

+-----+
|  _c0|
+-----+
|22509|
+-----+



In [67]:
sqlContext.sql("select id, geo.coordinates[0], coordinates.coordinates[1] from txs where geo is not null limit 1").show()

+------------------+----------+----------+
|                id|       _c1|       _c2|
+------------------+----------+----------+
|704455835645153280|25.4336309|25.4336309|
+------------------+----------+----------+



In [68]:
sqlContext.sql("select count(*) from txs where geo.coordinates[0]<>coordinates.coordinates[1]").show()

+---+
|_c0|
+---+
|  0|
+---+



In [69]:
sqlContext.sql("select count(*) from txs where geo.coordinates[1]<>coordinates.coordinates[0]").show()

+---+
|_c0|
+---+
|  0|
+---+



In [74]:
sqlContext.sql("select id,  coordinates.coordinates[1] as latitud ,coordinates.coordinates[0] as longitud \
               from txs where coordinates is not null limit 1").show()

+------------------+----------+------------+
|                id|   latitud|    longitud|
+------------------+----------+------------+
|704455835645153280|25.4336309|-100.9980081|
+------------------+----------+------------+



In [88]:
# No sabemos sobreescribir archivos. borramos todo.
! rm -r prueba.csv
sqlContext.sql("select id,  coordinates.coordinates[1] as latitud ,coordinates.coordinates[0] as longitud \
               from txs where coordinates is not null")\
            .map(lambda x: ",".join(map(str, x)))\
            .coalesce(1)\
            .saveAsTextFile("prueba.csv")

In [3]:
log_raw_hive_total = sqlContext.read.json("./febrero/*")

In [4]:
log_raw_hive_total.first()

Row(_corrupt_record=None, contributors=None, coordinates=None, created_at='Tue Feb 16 16:39:19 +0000 2016', entities=Row(hashtags=[Row(indices=[96, 103], text='Oaxaca'), Row(indices=[104, 115], text='twitteroax')], media=[Row(display_url='pic.twitter.com/Ge6PjAIMcA', expanded_url='http://twitter.com/CuartaPlana/status/699634169723383809/photo/1', id=699634168834097152, id_str='699634168834097152', indices=[116, 139], media_url='http://pbs.twimg.com/media/CbWZkdoVAAA6mWM.jpg', media_url_https='https://pbs.twimg.com/media/CbWZkdoVAAA6mWM.jpg', sizes=Row(large=Row(h=362, resize='fit', w=644), medium=Row(h=337, resize='fit', w=600), small=Row(h=191, resize='fit', w=340), thumb=Row(h=150, resize='crop', w=150)), source_status_id=None, source_status_id_str=None, source_user_id=None, source_user_id_str=None, type='photo', url='https://t.co/Ge6PjAIMcA')], symbols=[], urls=[Row(display_url='cuartaplana.com.mx/nota.php?n=8695', expanded_url='http://www.cuartaplana.com.mx/nota.php?n=8695', indice

In [7]:
log_raw_hive_total.count()

3738755

In [5]:
log_raw_hive_total.registerTempTable('tweets')

In [6]:
#! rm -rf tweets_febrero
sqlContext.sql("select id,  coordinates.coordinates[1] as latitud ,coordinates.coordinates[0] as longitud, \
            created_at from tweets where coordinates is not null")\
            .map(lambda x: ",".join(map(str, x)))\
            .coalesce(8)\
            .saveAsTextFile("tweets_febrero")